In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
# Load the dataset from an Excel file
data = pd.read_excel('gpt2_embeddings.xlsx')

# Assume the last column is the target variable
X = data.iloc[:, :-1]  # Features
y = data.iloc[:, -1]    # Target variable

# Split the data into training and test sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold, cross_validate

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
models = {
    "SVM": SVC(),
    "KNN": KNeighborsClassifier(),
    "Decision Tree": DecisionTreeClassifier(),
    "Gaussian Naive Bayes": GaussianNB(),
    "Random Forest": RandomForestClassifier(),
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "MLP": MLPClassifier(max_iter=1000, random_state=42),
    "XGBoost": XGBClassifier(eval_metric='logloss'),
    "CatBoost": CatBoostClassifier(iterations=1000, learning_rate=0.1, depth=6, verbose=0),
    "AdaBoost": AdaBoostClassifier(n_estimators=100, random_state=42),
    "Extra Trees": ExtraTreesClassifier(n_estimators=100, random_state=42),
    "Gradient Boosting": GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
}

# Evaluate models
results = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='macro')
    recall = recall_score(y_test, y_pred, average='macro')
    f1 = f1_score(y_test, y_pred, average='macro')

    results[name] = {
        'Accuracy': round(accuracy, 4),
        'Precision': round(precision, 4),
        'Recall': round(recall, 4),
        'F1 Score': round(f1, 4)
    }

# Print the results
for model_name, metrics in results.items():
    print(f"Model: {model_name}")
    for metric, value in metrics.items():
        print(f"  {metric}: {value}")

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Model: SVM
  Accuracy: 0.5893
  Precision: 0.5997
  Recall: 0.5802
  F1 Score: 0.5855
Model: KNN
  Accuracy: 0.5863
  Precision: 0.5912
  Recall: 0.5811
  F1 Score: 0.5842
Model: Decision Tree
  Accuracy: 0.4554
  Precision: 0.4499
  Recall: 0.4464
  F1 Score: 0.4477
Model: Gaussian Naive Bayes
  Accuracy: 0.5268
  Precision: 0.5293
  Recall: 0.539
  F1 Score: 0.5274
Model: Random Forest
  Accuracy: 0.5714
  Precision: 0.582
  Recall: 0.5564
  F1 Score: 0.5621
Model: Logistic Regression
  Accuracy: 0.5625
  Precision: 0.5699
  Recall: 0.5599
  F1 Score: 0.5628
Model: MLP
  Accuracy: 0.5804
  Precision: 0.5883
  Recall: 0.577
  F1 Score: 0.5806
Model: XGBoost
  Accuracy: 0.5744
  Precision: 0.5848
  Recall: 0.5609
  F1 Score: 0.5669
Model: CatBoost
  Accuracy: 0.5952
  Precision: 0.6089
  Recall: 0.5839
  F1 Score: 0.5904
Model: AdaBoost
  Accuracy: 0.5506
  Precision: 0.5607
  Recall: 0.5369
  F1 Score: 0.5427
Model: Extra Trees
  Accuracy: 0.5685
  Precision: 0.5773
  Recall: 0.5581
 

In [ ]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 4.8 MB/s eta 0:00:00


In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# Define the kNN model
knn = KNeighborsClassifier()

# Set up the parameter grid for RandomizedSearchCV
param_distributions = {
    'n_neighbors': [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'minkowski']
}

# Set up the RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=knn,
    param_distributions=param_distributions,
    n_iter=20,  # Number of random combinations to try
    cv=10,  # 10-fold cross-validation
    scoring='f1_macro',  # Change this to any other scoring metric if needed
    random_state=42,
    n_jobs=-1
)

# Fit the model to find the best parameters
random_search.fit(X_train, y_train)

# Output the best parameters
print("Best Parameters:", random_search.best_params_)

best_knn = random_search.best_estimator_

# Use StratifiedKFold for cross-validation
cv = StratifiedKFold(n_splits=10)

# Perform cross-validation on the best model using StratifiedKFold
cv_score = cross_validate(best_knn, X_train, y_train, cv=cv,
                          scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro'],
                          return_train_score=False)

# Collect and round the results
results = {
    'Accuracy Mean': round(cv_score['test_accuracy'].mean(), 4),
    'Accuracy STD': round(cv_score['test_accuracy'].std(), 4),
    'Precision Mean': round(cv_score['test_precision_macro'].mean(), 4),
    'Precision STD': round(cv_score['test_precision_macro'].std(), 4),
    'Recall Mean': round(cv_score['test_recall_macro'].mean(), 4),
    'Recall STD': round(cv_score['test_recall_macro'].std(), 4),
    'F1 Mean': round(cv_score['test_f1_macro'].mean(), 4),
    'F1 STD': round(cv_score['test_f1_macro'].std(), 4),
}

# Print the evaluation metrics
print("Evaluation Metrics:", results)


Best Parameters: {'weights': 'distance', 'n_neighbors': 15, 'metric': 'minkowski'}
Evaluation Metrics: {'Accuracy Mean': 0.6079, 'Accuracy STD': 0.0392, 'Precision Mean': 0.6175, 'Precision STD': 0.0394, 'Recall Mean': 0.6098, 'Recall STD': 0.0419, 'F1 Mean': 0.6102, 'F1 STD': 0.0397}


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold, cross_validate

# Define the Gradient Boosting model
gb_model = GradientBoostingClassifier()

# Set up the parameter grid for RandomizedSearchCV
param_distributions = {
    'n_estimators': [50, 100, 150, 200],
    'learning_rate': [0.01, 0.1, 0.2, 0.3],
    'max_depth': [3, 4, 5, 6],
    'subsample': [0.8, 1.0],
    'max_features': ['auto', 'sqrt', 'log2']
}

# Set up the RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=gb_model,
    param_distributions=param_distributions,
    n_iter=20,  # Number of random combinations to try
    cv=10,  # 10-fold cross-validation
    scoring='f1_macro',  # Change this to any other scoring metric if needed
    random_state=42,
    n_jobs=-1
)

# Fit the model to find the best parameters
random_search.fit(X_train, y_train)

# Output the best parameters
print("Best Parameters:", random_search.best_params_)

best_gb = random_search.best_estimator_

# Use StratifiedKFold for cross-validation
cv = StratifiedKFold(n_splits=10)

# Perform cross-validation on the best model using StratifiedKFold
cv_score = cross_validate(best_gb, X_train, y_train, cv=cv,
                          scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro'],
                          return_train_score=False)

# Collect and round the results
results = {
    'Accuracy Mean': round(cv_score['test_accuracy'].mean(), 4),
    'Accuracy STD': round(cv_score['test_accuracy'].std(), 4),
    'Precision Mean': round(cv_score['test_precision_macro'].mean(), 4),
    'Precision STD': round(cv_score['test_precision_macro'].std(), 4),
    'Recall Mean': round(cv_score['test_recall_macro'].mean(), 4),
    'Recall STD': round(cv_score['test_recall_macro'].std(), 4),
    'F1 Mean': round(cv_score['test_f1_macro'].mean(), 4),
    'F1 STD': round(cv_score['test_f1_macro'].std(), 4),
}

# Print the evaluation metrics
print("Evaluation Metrics:", results)


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
70 fits failed out of a total of 200.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
7 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 1466, in wrapper
    estimator._validate_params()
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/utils/_

Best Parameters: {'subsample': 0.8, 'n_estimators': 200, 'max_features': 'log2', 'max_depth': 4, 'learning_rate': 0.2}
Evaluation Metrics: {'Accuracy Mean': 0.6146, 'Accuracy STD': 0.0251, 'Precision Mean': 0.6237, 'Precision STD': 0.0242, 'Recall Mean': 0.6165, 'Recall STD': 0.0255, 'F1 Mean': 0.6175, 'F1 STD': 0.0232}


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
70 fits failed out of a total of 200.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
53 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 1466, in wrapper
    estimator._validate_params()
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/utils/

Best Parameters: {'subsample': 0.8, 'n_estimators': 200, 'max_features': 'log2', 'max_depth': 3, 'learning_rate': 0.1}
Evaluation Metrics: {'Accuracy Mean': 0.6273, 'Accuracy STD': 0.0277, 'Precision Mean': 0.6359, 'Precision STD': 0.0271, 'Recall Mean': 0.6263, 'Recall STD': 0.0276, 'F1 Mean': 0.6287, 'F1 STD': 0.0264}


In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold, cross_validate

# Define the Extra Trees model
extra_trees_model = ExtraTreesClassifier()

# Set up the parameter grid for RandomizedSearchCV
param_distributions = {
    'n_estimators': [50, 100, 150, 200],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [False, True]
}

# Set up the RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=extra_trees_model,
    param_distributions=param_distributions,
    n_iter=20,  # Number of random combinations to try
    cv=10,  # 10-fold cross-validation
    scoring='f1_macro',  # Change this to any other scoring metric if needed
    random_state=42,
    n_jobs=-1
)

# Fit the model to find the best parameters
random_search.fit(X_train, y_train)

# Output the best parameters
print("Best Parameters:", random_search.best_params_)

best_extra_trees = random_search.best_estimator_

# Use StratifiedKFold for cross-validation
cv = StratifiedKFold(n_splits=10)

# Perform cross-validation on the best model using StratifiedKFold
cv_score = cross_validate(best_extra_trees, X_train, y_train, cv=cv,
                          scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro'],
                          return_train_score=False)

# Collect and round the results
results = {
    'Accuracy Mean': round(cv_score['test_accuracy'].mean(), 4),
    'Accuracy STD': round(cv_score['test_accuracy'].std(), 4),
    'Precision Mean': round(cv_score['test_precision_macro'].mean(), 4),
    'Precision STD': round(cv_score['test_precision_macro'].std(), 4),
    'Recall Mean': round(cv_score['test_recall_macro'].mean(), 4),
    'Recall STD': round(cv_score['test_recall_macro'].std(), 4),
    'F1 Mean': round(cv_score['test_f1_macro'].mean(), 4),
    'F1 STD': round(cv_score['test_f1_macro'].std(), 4),
}

# Print the evaluation metrics
print("Evaluation Metrics:", results)


KeyboardInterrupt: 

In [ ]:
from sklearn.naive_bayes import GaussianNB

nb = GaussianNB()

# No hyperparameters to tune, so we can fit and evaluate directly
nb.fit(X_train, y_train)

# Perform cross-validation on the model
cv_score = cross_validate(nb, X_train, y_train, cv=10, scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro'], return_train_score=False)
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE

# Step 1: Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Step 2: Handle class imbalance using SMOTE
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)

# Step 3: Train the Naive Bayes model
nb = GaussianNB(var_smoothing=1e-9)  # Apply Laplace smoothing
nb.fit(X_train_resampled, y_train_resampled)

# Step 4: Evaluate using Stratified K-Fold Cross-Validation
skf = StratifiedKFold(n_splits=10)

cv_score = cross_validate(nb, X_train_resampled, y_train_resampled, cv=skf,
                          scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro'],
                          return_train_score=False)

# Collect and round the results
results = {
    'Accuracy Mean': round(cv_score['test_accuracy'].mean(), 4),
    'Accuracy STD': round(cv_score['test_accuracy'].std(), 4),
    'Precision Mean': round(cv_score['test_precision_macro'].mean(), 4),
    'Precision STD': round(cv_score['test_precision_macro'].std(), 4),
    'Recall Mean': round(cv_score['test_recall_macro'].mean(), 4),
    'Recall STD': round(cv_score['test_recall_macro'].std(), 4),
    'F1 Mean': round(cv_score['test_f1_macro'].mean(), 4),
    'F1 STD': round(cv_score['test_f1_macro'].std(), 4),
}

print("Naive Bayes Evaluation Metrics:", results)



Naive Bayes Evaluation Metrics: {'Accuracy Mean': 0.5403, 'Accuracy STD': 0.0224, 'Precision Mean': 0.5359, 'Precision STD': 0.0218, 'Recall Mean': 0.5403, 'Recall STD': 0.0224, 'F1 Mean': 0.5337, 'F1 STD': 0.023}


In [ ]:

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold

# Define the Random Forest model
rf = RandomForestClassifier(random_state=42)

# Set up the parameter grid for RandomizedSearchCV
param_distributions = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5,10],
    'min_samples_leaf': [1, 2],
    'max_features': ['sqrt'],
    'bootstrap': [True,False],
    'class_weight': ['balanced']
}

# Set up RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_distributions,
    n_iter=5,
    cv=StratifiedKFold(n_splits=5),
    scoring='f1_macro',
    random_state=42,
    n_jobs=-1
)

# Fit the RandomizedSearchCV
random_search.fit(X_train, y_train)

# Output best parameters and score
print("Best Parameters:", random_search.best_params_)

# Get the best model
best_rf = random_search.best_estimator_

# Perform cross-validation on the best Random Forest model
cv_score = cross_validate(best_rf, X_train, y_train, cv=5,
                          scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro'], return_train_score=False)

# Collect and round the results
results = {
    'Accuracy Mean': round(cv_score['test_accuracy'].mean(), 4),
    'Accuracy STD': round(cv_score['test_accuracy'].std(), 4),
    'Precision Mean': round(cv_score['test_precision_macro'].mean(), 4),
    'Precision STD': round(cv_score['test_precision_macro'].std(), 4),
    'Recall Mean': round(cv_score['test_recall_macro'].mean(), 4),
    'Recall STD': round(cv_score['test_recall_macro'].std(), 4),
    'F1 Mean': round(cv_score['test_f1_macro'].mean(), 4),
    'F1 STD': round(cv_score['test_f1_macro'].std(), 4),
}

# Print the evaluation metrics
print("Evaluation Metrics:", results)


Best Parameters: {'n_estimators': 100, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': 10, 'class_weight': 'balanced', 'bootstrap': True}
Evaluation Metrics: {'Accuracy Mean': 0.6161, 'Accuracy STD': 0.0169, 'Precision Mean': 0.6221, 'Precision STD': 0.0149, 'Recall Mean': 0.6202, 'Recall STD': 0.0187, 'F1 Mean': 0.6197, 'F1 STD': 0.016}


In [ ]:
from sklearn.linear_model import LogisticRegression

# Define the Logistic Regression model
log_reg = LogisticRegression(solver='liblinear', random_state=42)

# Set up the parameter grid for RandomizedSearchCV
param_distributions = {
    'penalty': ['l1', 'l2'],
    'C': [0.01, 0.1, 1.0, 10.0],
}

# Set up the RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=log_reg,
    param_distributions=param_distributions,
    n_iter=5,
    cv=10,
    scoring='f1_macro',
    random_state=42,
    n_jobs=-1
)

random_search.fit(X_train, y_train)

print("Logistic Regression Best Parameters:", random_search.best_params_)

# Perform cross-validation on the best model
best_log_reg = random_search.best_estimator_
cv_score = cross_validate(best_log_reg, X_train, y_train, cv=10, scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro'], return_train_score=False)

# Collect and round the results
results = {
    'Accuracy Mean': round(cv_score['test_accuracy'].mean(), 4),
    'Accuracy STD': round(cv_score['test_accuracy'].std(), 4),
    'Precision Mean': round(cv_score['test_precision_macro'].mean(), 4),
    'Precision STD': round(cv_score['test_precision_macro'].std(), 4),
    'Recall Mean': round(cv_score['test_recall_macro'].mean(), 4),
    'Recall STD': round(cv_score['test_recall_macro'].std(), 4),
    'F1 Mean': round(cv_score['test_f1_macro'].mean(), 4),
    'F1 STD': round(cv_score['test_f1_macro'].std(), 4),
}

print("Logistic Regression Evaluation Metrics:", results)


Logistic Regression Best Parameters: {'penalty': 'l2', 'C': 0.01}
Logistic Regression Evaluation Metrics: {'Accuracy Mean': 0.6034, 'Accuracy STD': 0.0336, 'Precision Mean': 0.6035, 'Precision STD': 0.0334, 'Recall Mean': 0.6102, 'Recall STD': 0.0358, 'F1 Mean': 0.6044, 'F1 STD': 0.0328}


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Load the dataset from an Excel file
data = pd.read_excel('gpt2_embeddings.xlsx')

# Assume the last column is the target variable
X = data.iloc[:, :-1]  # Features
y = data.iloc[:, -1]    # Target variable

# Split the data into training and test sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold, cross_validate
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE

# Step 1: Feature Scaling (important for many models)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Step 2: Handle class imbalance using SMOTE (if needed)
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)

# Step 3: Define the AdaBoost model with a shallow decision tree as the base estimator
base_estimator = DecisionTreeClassifier(max_depth=3, random_state=42)

ada = AdaBoostClassifier(
    estimator=base_estimator,
    random_state=42
)

# Step 4: Set up the parameter grid for RandomizedSearchCV
param_distributions = {
    'n_estimators': [50, 100, 200],  # Number of boosting rounds (trees)
    'learning_rate': [0.01, 0.1, 1.0],  # How much each estimator contributes
    'estimator__max_depth': [3, 5, 7]  # Max depth of the decision tree (base estimator)
}

# Step 5: Set up RandomizedSearchCV with StratifiedKFold for cross-validation
skf = StratifiedKFold(n_splits=5)

random_search = RandomizedSearchCV(
    estimator=ada,
    param_distributions=param_distributions,
    n_iter=5,
    cv=skf,
    scoring='f1_macro',
    random_state=42,
    n_jobs=-1
)

# Step 6: Fit the model with the best parameters
random_search.fit(X_train_resampled, y_train_resampled)

# Print the best parameters found
print("AdaBoost Best Parameters:", random_search.best_params_)

# Step 7: Perform cross-validation on the best AdaBoost model
best_ada = random_search.best_estimator_
cv_score = cross_validate(
    best_ada, X_train_resampled, y_train_resampled, cv=skf, scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro'], return_train_score=False
)
# Step 8: Collect and round the results
results = {
    'Accuracy Mean': round(cv_score['test_accuracy'].mean(), 4),
    'Accuracy STD': round(cv_score['test_accuracy'].std(), 4),
    'Precision Mean': round(cv_score['test_precision_macro'].mean(), 4),
    'Precision STD': round(cv_score['test_precision_macro'].std(), 4),
    'Recall Mean': round(cv_score['test_recall_macro'].mean(), 4),
    'Recall STD': round(cv_score['test_recall_macro'].std(), 4),
    'F1 Mean': round(cv_score['test_f1_macro'].mean(), 4),
    'F1 STD': round(cv_score['test_f1_macro'].std(), 4),
}

# Print the evaluation metrics
print("AdaBoost Evaluation Metrics:", results)


In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold, cross_validate

# Load the dataset
data = pd.read_excel('gpt2_embeddings.xlsx')

# Assume the last column is the target variable
X = data.iloc[:, :-1]  # Features
y = data.iloc[:, -1]   # Target

# Scale the features
X_scaled = StandardScaler().fit_transform(X)

# Apply PCA to retain 95% of variance
pca = PCA(n_components=0.9999)
X_pca = pca.fit_transform(X_scaled)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, random_state=42)



In [ ]:
# Define the SVM model
svm = SVC()

# Set up the parameter grid for RandomizedSearchCV
param_distributions = {
    'C': [0.1, 1, 10, 100],
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': ['scale', 'auto'],
    'degree': [2, 3, 4]  # Applicable only for polynomial kernel
}

# Set up the RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=svm,
    param_distributions=param_distributions,
    n_iter=20,  # Number of random combinations to try
    cv=10,  # 10-fold cross-validation
    scoring='f1_macro',  # Change this to any other scoring metric if needed
    random_state=42,
    n_jobs=-1  # Use all available cores
)

# Fit the random search model
random_search.fit(X_train, y_train)

# Print the best parameters found
print("Best Parameters:", random_search.best_params_)

# Get the best estimator
best_svm = random_search.best_estimator_

# Perform cross-validation on the best model
cv_score = cross_validate(best_svm, X_train, y_train, cv=10, scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro'], return_train_score=False)

# Collect and round the results
results = {
    'Accuracy Mean': round(cv_score['test_accuracy'].mean(), 4),
    'Accuracy STD': round(cv_score['test_accuracy'].std(), 4),
    'Precision Mean': round(cv_score['test_precision_macro'].mean(), 4),
    'Precision STD': round(cv_score['test_precision_macro'].std(), 4),
    'Recall Mean': round(cv_score['test_recall_macro'].mean(), 4),
    'Recall STD': round(cv_score['test_recall_macro'].std(), 4),
    'F1 Mean': round(cv_score['test_f1_macro'].mean(), 4),
    'F1 STD': round(cv_score['test_f1_macro'].std(), 4),
}

# Print the evaluation metrics
print("Evaluation Metrics:", results)

/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Best Parameters: {'kernel': 'rbf', 'gamma': 'auto', 'degree': 3, 'C': 1}
Evaluation Metrics: {'Accuracy Mean': 0.6192, 'Accuracy STD': 0.0517, 'Precision Mean': 0.6345, 'Precision STD': 0.0544, 'Recall Mean': 0.616, 'Recall STD': 0.0521, 'F1 Mean': 0.6217, 'F1 STD': 0.0534}


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold, cross_validate

# Define the Gradient Boosting model
gb_model = GradientBoostingClassifier()

# Set up the parameter grid for RandomizedSearchCV
param_distributions = {
    'n_estimators': [50, 100, 150, 200],
    'learning_rate': [0.01, 0.1, 0.2, 0.3],
    'max_depth': [3, 4, 5, 6],
    'subsample': [0.8, 1.0],
    'max_features': ['auto', 'sqrt', 'log2']
}

# Set up the RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=gb_model,
    param_distributions=param_distributions,
    n_iter=20,  # Number of random combinations to try
    cv=10,  # 10-fold cross-validation
    scoring='f1_macro',  # Change this to any other scoring metric if needed
    random_state=42,
    n_jobs=-1
)

# Fit the model to find the best parameters
random_search.fit(X_train, y_train)

# Output the best parameters
print("Best Parameters:", random_search.best_params_)

best_gb = random_search.best_estimator_

# Use StratifiedKFold for cross-validation
cv = StratifiedKFold(n_splits=10)

# Perform cross-validation on the best model using StratifiedKFold
cv_score = cross_validate(best_gb, X_train, y_train, cv=cv,
                          scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro'],
                          return_train_score=False)

# Collect and round the results
results = {
    'Accuracy Mean': round(cv_score['test_accuracy'].mean(), 4),
    'Accuracy STD': round(cv_score['test_accuracy'].std(), 4),
    'Precision Mean': round(cv_score['test_precision_macro'].mean(), 4),
    'Precision STD': round(cv_score['test_precision_macro'].std(), 4),
    'Recall Mean': round(cv_score['test_recall_macro'].mean(), 4),
    'Recall STD': round(cv_score['test_recall_macro'].std(), 4),
    'F1 Mean': round(cv_score['test_f1_macro'].mean(), 4),
    'F1 STD': round(cv_score['test_f1_macro'].std(), 4),
}

# Print the evaluation metrics
print("Evaluation Metrics:", results)


In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold, cross_validate

# Define the Extra Trees model
extra_trees_model = ExtraTreesClassifier()

# Set up the parameter grid for RandomizedSearchCV
param_distributions = {
    'n_estimators': [50, 100, 150, 200],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [False, True]
}

# Set up the RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=extra_trees_model,
    param_distributions=param_distributions,
    n_iter=20,  # Number of random combinations to try
    cv=10,  # 10-fold cross-validation
    scoring='f1_macro',  # Change this to any other scoring metric if needed
    random_state=42,
    n_jobs=-1
)

# Fit the model to find the best parameters
random_search.fit(X_train, y_train)

# Output the best parameters
print("Best Parameters:", random_search.best_params_)

best_extra_trees = random_search.best_estimator_

# Use StratifiedKFold for cross-validation
cv = StratifiedKFold(n_splits=10)

# Perform cross-validation on the best model using StratifiedKFold
cv_score = cross_validate(best_extra_trees, X_train, y_train, cv=cv,
                          scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro'],
                          return_train_score=False)

# Collect and round the results
results = {
    'Accuracy Mean': round(cv_score['test_accuracy'].mean(), 4),
    'Accuracy STD': round(cv_score['test_accuracy'].std(), 4),
    'Precision Mean': round(cv_score['test_precision_macro'].mean(), 4),
    'Precision STD': round(cv_score['test_precision_macro'].std(), 4),
    'Recall Mean': round(cv_score['test_recall_macro'].mean(), 4),
    'Recall STD': round(cv_score['test_recall_macro'].std(), 4),
    'F1 Mean': round(cv_score['test_f1_macro'].mean(), 4),
    'F1 STD': round(cv_score['test_f1_macro'].std(), 4),
}

# Print the evaluation metrics
print("Evaluation Metrics:", results)


In [ ]:
# Define the kNN model
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold, cross_validate

knn = KNeighborsClassifier()

# Set up the parameter grid for RandomizedSearchCV
param_distributions = {
    'n_neighbors': [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'minkowski']
}

# Set up the RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=knn,
    param_distributions=param_distributions,
    n_iter=20,  # Number of random combinations to try
    cv=10,  # 10-fold cross-validation
    scoring='f1_macro',  # Change this to any other scoring metric if needed
    random_state=42,
    n_jobs=-1
)

# Fit the model to find the best parameters
random_search.fit(X_train, y_train)

# Output the best parameters
print("Best Parameters:", random_search.best_params_)

best_knn = random_search.best_estimator_

# Use StratifiedKFold for cross-validation
cv = StratifiedKFold(n_splits=10)

# Perform cross-validation on the best model using StratifiedKFold
cv_score = cross_validate(best_knn, X_train, y_train, cv=cv,
                          scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro'],
                          return_train_score=True)

# Collect and round the results
results = {
    'Accuracy Mean': round(cv_score['test_accuracy'].mean(), 4),
    'Accuracy STD': round(cv_score['test_accuracy'].std(), 4),
    'Training Accuracy Mean': round(cv_score['train_accuracy'].mean(), 4),  # Added training accuracy
    'Training Accuracy STD': round(cv_score['train_accuracy'].std(), 4),
    'Precision Mean': round(cv_score['test_precision_macro'].mean(), 4),
    'Precision STD': round(cv_score['test_precision_macro'].std(), 4),
    'Recall Mean': round(cv_score['test_recall_macro'].mean(), 4),
    'Recall STD': round(cv_score['test_recall_macro'].std(), 4),
    'F1 Mean': round(cv_score['test_f1_macro'].mean(), 4),
    'F1 STD': round(cv_score['test_f1_macro'].std(), 4),
}

# Print the evaluation metrics
print("Evaluation Metrics:", results)

Best Parameters: {'weights': 'uniform', 'n_neighbors': 5, 'metric': 'euclidean'}
Evaluation Metrics: {'Accuracy Mean': 0.6118, 'Accuracy STD': 0.0584, 'Training Accuracy Mean': 0.7319, 'Training Accuracy STD': 0.0093, 'Precision Mean': 0.6179, 'Precision STD': 0.061, 'Recall Mean': 0.6128, 'Recall STD': 0.0583, 'F1 Mean': 0.6133, 'F1 STD': 0.0593}


In [ ]:
from sklearn.svm import SVC

# Define the SVM model
svm = SVC()

# Set up the parameter grid for RandomizedSearchCV
param_distributions = {
    'C': [0.1, 1, 10, 100],
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': ['scale', 'auto'],
    'degree': [2, 3, 4]  # Applicable only for polynomial kernel
}

# Set up the RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=svm,
    param_distributions=param_distributions,
    n_iter=20,  # Number of random combinations to try
    cv=10,  # 10-fold cross-validation
    scoring='f1_macro',  # Change this to any other scoring metric if needed
    random_state=42,
    n_jobs=-1  # Use all available cores
)

# Fit the random search model
random_search.fit(X_train, y_train)

# Print the best parameters found
print("Best Parameters:", random_search.best_params_)

best_svm = random_search.best_estimator_

# Use StratifiedKFold for cross-validation
cv = StratifiedKFold(n_splits=10)

# Perform cross-validation on the best model using StratifiedKFold
cv_score = cross_validate(best_svm, X_train, y_train, cv=cv,
                          scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro'],
                          return_train_score=True)

# Collect and round the results
results = {
    'Accuracy Mean': round(cv_score['test_accuracy'].mean(), 4),
    'Accuracy STD': round(cv_score['test_accuracy'].std(), 4),
    'Training Accuracy Mean': round(cv_score['train_accuracy'].mean(), 4),  # Added training accuracy
    'Training Accuracy STD': round(cv_score['train_accuracy'].std(), 4),
    'Precision Mean': round(cv_score['test_precision_macro'].mean(), 4),
    'Precision STD': round(cv_score['test_precision_macro'].std(), 4),
    'Recall Mean': round(cv_score['test_recall_macro'].mean(), 4),
    'Recall STD': round(cv_score['test_recall_macro'].std(), 4),
    'F1 Mean': round(cv_score['test_f1_macro'].mean(), 4),
    'F1 STD': round(cv_score['test_f1_macro'].std(), 4),
}

# Print the evaluation metrics
print("Evaluation Metrics:", results)

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold, cross_validate

nb = GaussianNB()

# No hyperparameters to tune, so we can fit and evaluate directly
nb.fit(X_train, y_train)

# Perform cross-validation on the model
cv_score = cross_validate(nb, X_train, y_train, cv=10, scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro'], return_train_score=False)
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE

# Step 1: Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Step 2: Handle class imbalance using SMOTE
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)

# Step 3: Train the Naive Bayes model
nb = GaussianNB(var_smoothing=1e-9)  # Apply Laplace smoothing
nb.fit(X_train_resampled, y_train_resampled)

# Step 4: Evaluate using Stratified K-Fold Cross-Validation
skf = StratifiedKFold(n_splits=10)

cv_score = cross_validate(nb, X_train_resampled, y_train_resampled, cv=skf,
                          scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro'],
                          return_train_score=False)

# Collect and round the results
results = {
    'Accuracy Mean': round(cv_score['test_accuracy'].mean(), 4),
    'Accuracy STD': round(cv_score['test_accuracy'].std(), 4),
    'Precision Mean': round(cv_score['test_precision_macro'].mean(), 4),
    'Precision STD': round(cv_score['test_precision_macro'].std(), 4),
    'Recall Mean': round(cv_score['test_recall_macro'].mean(), 4),
    'Recall STD': round(cv_score['test_recall_macro'].std(), 4),
    'F1 Mean': round(cv_score['test_f1_macro'].mean(), 4),
    'F1 STD': round(cv_score['test_f1_macro'].std(), 4),
}

print("Naive Bayes Evaluation Metrics:", results)



Naive Bayes Evaluation Metrics: {'Accuracy Mean': 0.4694, 'Accuracy STD': 0.0419, 'Precision Mean': 0.4764, 'Precision STD': 0.0425, 'Recall Mean': 0.4694, 'Recall STD': 0.0424, 'F1 Mean': 0.444, 'F1 STD': 0.0415}


In [ ]:

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold

# Define the Random Forest model
rf = RandomForestClassifier(random_state=42)

# Set up the parameter grid for RandomizedSearchCV
param_distributions = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5,10],
    'min_samples_leaf': [1, 2],
    'max_features': ['sqrt'],
    'bootstrap': [True,False],
    'class_weight': ['balanced']
}

# Set up RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_distributions,
    n_iter=20,
    cv=StratifiedKFold(n_splits=10),
    scoring='f1_macro',
    random_state=42,
    n_jobs=-1
)

# Fit the RandomizedSearchCV
random_search.fit(X_train, y_train)

# Output best parameters and score
print("Best Parameters:", random_search.best_params_)

# Get the best model
best_rf = random_search.best_estimator_

# Perform cross-validation on the best Random Forest model
cv_score = cross_validate(best_rf, X_train, y_train, cv=10,
                          scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro'], return_train_score=False)

# Collect and round the results
results = {
    'Accuracy Mean': round(cv_score['test_accuracy'].mean(), 4),
    'Accuracy STD': round(cv_score['test_accuracy'].std(), 4),
    'Precision Mean': round(cv_score['test_precision_macro'].mean(), 4),
    'Precision STD': round(cv_score['test_precision_macro'].std(), 4),
    'Recall Mean': round(cv_score['test_recall_macro'].mean(), 4),
    'Recall STD': round(cv_score['test_recall_macro'].std(), 4),
    'F1 Mean': round(cv_score['test_f1_macro'].mean(), 4),
    'F1 STD': round(cv_score['test_f1_macro'].std(), 4),
}

# Print the evaluation metrics
print("Evaluation Metrics:", results)


Best Parameters: {'n_estimators': 100, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': 10, 'class_weight': 'balanced', 'bootstrap': False}
Evaluation Metrics: {'Accuracy Mean': 0.6035, 'Accuracy STD': 0.0419, 'Precision Mean': 0.6086, 'Precision STD': 0.0424, 'Recall Mean': 0.6043, 'Recall STD': 0.0431, 'F1 Mean': 0.6047, 'F1 STD': 0.0423}


In [ ]:
from sklearn.linear_model import LogisticRegression

# Define the Logistic Regression model
log_reg = LogisticRegression(solver='liblinear', random_state=42)

# Set up the parameter grid for RandomizedSearchCV
param_distributions = {
    'penalty': ['l1', 'l2'],
    'C': [0.01, 0.1, 1.0, 10.0],
}

# Set up the RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=log_reg,
    param_distributions=param_distributions,
    n_iter=20,
    cv=10,
    scoring='f1_macro',
    random_state=42,
    n_jobs=-1
)

random_search.fit(X_train, y_train)

print("Logistic Regression Best Parameters:", random_search.best_params_)

# Perform cross-validation on the best model
best_log_reg = random_search.best_estimator_
cv_score = cross_validate(best_log_reg, X_train, y_train, cv=10, scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro'], return_train_score=False)

# Collect and round the results
results = {
    'Accuracy Mean': round(cv_score['test_accuracy'].mean(), 4),
    'Accuracy STD': round(cv_score['test_accuracy'].std(), 4),
    'Precision Mean': round(cv_score['test_precision_macro'].mean(), 4),
    'Precision STD': round(cv_score['test_precision_macro'].std(), 4),
    'Recall Mean': round(cv_score['test_recall_macro'].mean(), 4),
    'Recall STD': round(cv_score['test_recall_macro'].std(), 4),
    'F1 Mean': round(cv_score['test_f1_macro'].mean(), 4),
    'F1 STD': round(cv_score['test_f1_macro'].std(), 4),
}

print("Logistic Regression Evaluation Metrics:", results)


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold, cross_validate
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE  # For handling class imbalance

# Optional: Scaling the features (this isn't strictly necessary for Decision Trees but may help)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Optional: Handling class imbalance using SMOTE
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)

# Define the Decision Tree model
dtree = DecisionTreeClassifier(random_state=42, class_weight='balanced')  # Use class_weight='balanced' to handle class imbalance

# Set up the parameter grid for RandomizedSearchCV
param_distributions = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 5, 10, 15, 20, 25],
    'min_samples_split': [2, 5, 10, 20],
    'min_samples_leaf': [1, 2, 4, 6, 10],
    'max_features': [None, 'sqrt', 'log2'],
}

# Set up the RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=dtree,
    param_distributions=param_distributions,
    n_iter=20,
    cv=StratifiedKFold(n_splits=10),
    scoring='f1_macro',
    random_state=42,
    n_jobs=-1
)

random_search.fit(X_train_resampled, y_train_resampled)

print("Best Parameters:", random_search.best_params_)

best_dtree = random_search.best_estimator_

cv_score = cross_validate(best_dtree, X_train_resampled, y_train_resampled, cv=StratifiedKFold(n_splits=10),
                          scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro'], return_train_score=False)

results = {
    'Accuracy Mean': round(cv_score['test_accuracy'].mean(), 4),
    'Accuracy STD': round(cv_score['test_accuracy'].std(), 4),
    'Precision Mean': round(cv_score['test_precision_macro'].mean(), 4),
    'Precision STD': round(cv_score['test_precision_macro'].std(), 4),
    'Recall Mean': round(cv_score['test_recall_macro'].mean(), 4),
    'Recall STD': round(cv_score['test_recall_macro'].std(), 4),
    'F1 Mean': round(cv_score['test_f1_macro'].mean(), 4),
    'F1 STD': round(cv_score['test_f1_macro'].std(), 4),
}

print("Evaluation Metrics:",results)

In [ ]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(max_iter=2000, random_state=42)

param_distributions = {
    'hidden_layer_sizes': [(50,), (100,)],
    'activation': ['logistic', 'tanh', 'relu'],
    'solver': ['adam', 'sgd'],
    'learning_rate': ['constant', 'adaptive'],
}

random_search = RandomizedSearchCV(
    estimator=mlp,
    param_distributions=param_distributions,
    n_iter=20,
    cv=10,
    scoring='f1_macro',
    random_state=42,
    n_jobs=-1
)

random_search.fit(X_train, y_train)

print("MLP Best Parameters:", random_search.best_params_)

best_mlp = random_search.best_estimator_
cv_score = cross_validate(best_mlp, X_train, y_train, cv=10, scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro'], return_train_score=False)

results = {
    'Accuracy Mean': round(cv_score['test_accuracy'].mean(), 4),
    'Accuracy STD': round(cv_score['test_accuracy'].std(), 4),
    'Precision Mean': round(cv_score['test_precision_macro'].mean(), 4),
    'Precision STD': round(cv_score['test_precision_macro'].std(), 4),
    'Recall Mean': round(cv_score['test_recall_macro'].mean(), 4),
    'Recall STD': round(cv_score['test_recall_macro'].std(), 4),
    'F1 Mean': round(cv_score['test_f1_macro'].mean(), 4),
    'F1 STD': round(cv_score['test_f1_macro'].std(), 4),
}

print("MLP Evaluation Metrics:",results)

MLP Best Parameters: {'solver': 'sgd', 'learning_rate': 'constant', 'hidden_layer_sizes': (100,), 'activation': 'relu'}
MLP Evaluation Metrics: {'Accuracy Mean': 0.6199, 'Accuracy STD': 0.0417, 'Precision Mean': 0.6245, 'Precision STD': 0.0424, 'Recall Mean': 0.623, 'Recall STD': 0.0419, 'F1 Mean': 0.6219, 'F1 STD': 0.0419}


In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold

xgb = XGBClassifier(random_state=42)

param_distributions = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01,  0.1],
    'subsample': [ 0.8, 1.0],
    'colsample_bytree': [ 0.8, 1.0],
    'gamma': [0, 1],
    'class_weight': ['balanced']
}

random_search = RandomizedSearchCV(
    estimator=xgb,
    param_distributions=param_distributions,
    n_iter=20,
    cv=StratifiedKFold(n_splits=10),
    scoring='f1_macro',
    random_state=42,
    n_jobs=-1
)

random_search.fit(X_train, y_train)

print("Best Parameters:", random_search.best_params_)

best_xgb = random_search.best_estimator_

cv_score = cross_validate(best_xgb, X_train, y_train, cv=10,
                          scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro'], return_train_score=False)

results = {
    'Accuracy Mean': round(cv_score['test_accuracy'].mean(), 4),
    'Accuracy STD': round(cv_score['test_accuracy'].std(), 4),
    'Precision Mean': round(cv_score['test_precision_macro'].mean(), 4),
    'Precision STD': round(cv_score['test_precision_macro'].std(), 4),
    'Recall Mean': round(cv_score['test_recall_macro'].mean(), 4),
    'Recall STD': round(cv_score['test_recall_macro'].std(), 4),
    'F1 Mean': round(cv_score['test_f1_macro'].mean(), 4),
    'F1 STD': round(cv_score['test_f1_macro'].std(), 4),
}

print("Evaluation Metrics:",results)

KeyboardInterrupt: 

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold, cross_validate
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)
base_estimator = DecisionTreeClassifier(max_depth=3, random_state=42)

ada = AdaBoostClassifier(
    estimator=base_estimator,
    random_state=42
)

param_distributions = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 1.0],
    'estimator__max_depth': [3, 5, 7]
}

skf = StratifiedKFold(n_splits=5)

random_search = RandomizedSearchCV(
    estimator=ada,
    param_distributions=param_distributions,
    n_iter=10,
    cv=skf,
    scoring='f1_macro',
    random_state=42,
    n_jobs=-1
)

random_search.fit(X_train_resampled, y_train_resampled)

print("AdaBoost Best Parameters:", random_search.best_params_)

best_ada = random_search.best_estimator_
cv_score = cross_validate(
    best_ada, X_train_resampled, y_train_resampled, cv=skf, scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro'], return_train_score=False
)
results = {
    'Accuracy Mean': round(cv_score['test_accuracy'].mean(), 4),
    'Accuracy STD': round(cv_score['test_accuracy'].std(), 4),
    'Precision Mean': round(cv_score['test_precision_macro'].mean(), 4),
    'Precision STD': round(cv_score['test_precision_macro'].std(), 4),
    'Recall Mean': round(cv_score['test_recall_macro'].mean(), 4),
    'Recall STD': round(cv_score['test_recall_macro'].std(), 4),
    'F1 Mean': round(cv_score['test_f1_macro'].mean(), 4),
    'F1 STD': round(cv_score['test_f1_macro'].std(), 4),
}

print("AdaBoost Evaluation Metrics:",results)

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


AdaBoost Best Parameters: {'n_estimators': 200, 'learning_rate': 1.0, 'estimator__max_depth': 5}


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/usr/local/lib/python3.1

AdaBoost Evaluation Metrics: {'Accuracy Mean': 0.642, 'Accuracy STD': 0.0797, 'Precision Mean': 0.6611, 'Precision STD': 0.0681, 'Recall Mean': 0.6421, 'Recall STD': 0.0797, 'F1 Mean': 0.6443, 'F1 STD': 0.0744}
